# PaddleX

## 1.客户端图像分类

### 1.1数据选择

<img src="https://ai-studio-static-online.cdn.bcebos.com/1c1dc0d6f757404cb461299bc4061c77e7851379511a400684b427d13237d206" width="600" />


### 1.2参数配置

<img src="https://ai-studio-static-online.cdn.bcebos.com/67f20c83adfc48ea99b8d4e2967ece607584b6e794c24d46aa5df9d0fcaca363" width="600" />


### 1.3模型训练

<img src="https://ai-studio-static-online.cdn.bcebos.com/28c5987924b84a4fac7c5af8076cc936ab8fb5b9270e41f3be5dfb86477ecf0b" width="600" />


### 1.4测试图片

<img src="https://ai-studio-static-online.cdn.bcebos.com/54c5a20ea19e4f7eb36360694bbf4397454a72e3f2ad4b5d81bcfdb67e74baad" width="600" />


## 2.客户端图像分类

In [ ]:
!unzip -oq /home/aistudio/data/data81942/00.zip

In [ ]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 102kB 19.7MB/s ta 0:00:01
     |████████████████████████████████| 1.0MB 13.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 70.5MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278369 sha256=1bbca2ad58aaf47537f2eaacd24043f0049fab5e950e1d2f2f564b2c7537864b
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


In [ ]:
!paddlex --data_conversion --source labelme --to MSCOCO \
        --pics 00 \
        --annotations 00 \
        --save_dir dataset_coco

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
!paddlex --split_dataset --format COCO --dataset_dir dataset_coco --val_value 0.2 --test_value 0.1


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is depre

In [ ]:
from paddlex.det import transforms

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.Normalize(),
    transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(), transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])


In [ ]:
import paddlex as pdx

train_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/val.json',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2021-04-21 19:10:40 [INFO]	Starting to read file list from dataset...
2021-04-21 19:10:40 [INFO]	293 samples in file dataset_coco/train.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2021-04-21 19:10:40 [INFO]	Starting to read file list from dataset...
2021-04-21 19:10:40 [INFO]	83 samples in file dataset_coco/val.json


In [7]:
num_classes = len(train_dataset.labels) + 1

model = pdx.det.MaskRCNN(num_classes=num_classes, backbone='ResNet50')

model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.00125,
    warmup_steps=10,
    lr_decay_epochs=[8, 11],
    save_dir='output/mask_rcnn_r50_fpn',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
2021-04-21 19:11:13,012 - INFO - If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-pad

2021-04-21 19:11:13 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


100%|██████████| 100310/100310 [00:06<00:00, 15145.95KB/s]


2021-04-21 19:11:20 [INFO]	Decompressing output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.tar...
2021-04-21 19:11:25 [INFO]	Load pretrain weights from output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.
2021-04-21 19:11:26 [INFO]	There are 265 varaibles in output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained are loaded.
2021-04-21 19:11:34 [INFO]	[TRAIN] Epoch=1/100, Step=2/73, loss=3.117201, loss_cls=0.713111, loss_bbox=0.0, loss_mask=1.336037, loss_rpn_cls=0.692118, loss_rpn_bbox=0.375935, lr=0.0005, time_each_step=3.78s, eta=9:52:24
2021-04-21 19:11:35 [INFO]	[TRAIN] Epoch=1/100, Step=4/73, loss=2.03852, loss_cls=0.56796, loss_bbox=0.0, loss_mask=0.548011, loss_rpn_cls=0.694677, loss_rpn_bbox=0.227872, lr=0.000667, time_each_step=2.24s, eta=5:50:49
2021-04-21 19:11:36 [INFO]	[TRAIN] Epoch=1/100, Step=6/73, loss=1.908709, loss_cls=0.478807, loss_bbox=0.0, loss_mask=0.548671, loss_rpn_cls=0.694603, loss_rpn_bbox=0.186628, lr=0.000833, time_each_step=1.69s, eta=4:2

100%|██████████| 83/83 [00:17<00:00,  4.84it/s]


...

creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.288
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.782
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.288
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.330
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

100%|██████████| 83/83 [00:15<00:00,  5.48it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.282
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.765
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.152
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.510
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.510
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

Process Process-230:
Process Process-228:
Process Process-231:
Process Process-232:
Process Process-226:
Traceback (most recent call last):
Process Process-227:
Process Process-225:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/multiprocessing/process.py", line 297, in _

KeyboardInterrupt: 

In [8]:
model = pdx.load_model('output/mask_rcnn_r50_fpn/best_model')
image_name = 'dataset_coco/JPEGImages/lhgd295.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5)

2021-04-21 19:26:30 [INFO]	Model[MaskRCNN] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterator):


2021-04-21 19:26:31 [INFO]	The visualized result is saved as ./visualize_lhgd295.jpg


![](https://ai-studio-static-online.cdn.bcebos.com/29090b6ea14f4f2e931f0d4abfe613452d3a47f8a1f3477bb652657bc2a75b9c)
